In [1]:
using GraphPPL, ReactiveMP, Distributions
# using Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1662


In [16]:
function f(x)
    return x
end

function f_inv(x)
    return x
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x) where {meta=ET(f_inv)}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [17]:
imarginals = (x = MvNormal(zeros(2), diageye(2)), )

result = inference(model = Model(NKS), data=(y2=1.0,), free_energy=false)

"out" = "out"
f(ones(2)) = [1.0, 1.0]
"in" = "in"


Inference results:
-----------------------------------------
y1 = NormalWeightedMeanPrecision{Float64}(xi=2.0, w=2.5)
x  = MvNormalWeightedMeanPrecision(
xi: [0.6666666666666665, 0.0]
Λ: [1.6666666666676...


In [18]:
function f(x, θ)
    return x .+ θ
end

function f_inv_x(x)
    return randn(length(θ))
end

function f_inv_θ(x)
    return randn(length(x))
end

c = randn(2);

@model function NKS()
    y2 = datavar(Float64)
    c = zeros(2); c[1] = 1.0;

    θ ~ MvNormal(μ=ones(2), Λ=diageye(2))
    x ~ MvNormal(μ=zeros(2), Λ=diageye(2))
    z ~ f(x, θ) where {meta=ET([f_inv_x, f_inv_θ])}
    y1 ~ Normal(μ=dot(z, c), σ²=1.0)
    y2 ~ Normal(μ=y1, σ²=0.5)
end

In [18]:
mean_cov(result.posteriors[:x])

([0.39999999999975994, 0.0], [0.59999999999964 -0.0; -0.0 0.9999999999989999])

In [47]:
function g(x)
    return x^2
end

function g_inv(x)
    return sqrt(x)
end

@model function NKS2(n)
    y = datavar(Float64, n)

    # γ ~ Gamma(α=1.0, β=1.0)
    x ~ Normal(μ = 0.0, var = 100)
    z ~ g(x) where { meta = ET(g_inv) }
    
    y .~ Normal(μ = z, var = 0.5)
end

# constraints = @constraints begin
#     q(x, γ) = q(x)q(γ)
# end

# imarginals = (x = MvNormal(zeros(2), diageye(2)), γ=Gamma(1.0, 1.0))

result = inference(model = Model(NKS2, 100), data=(y = 4 * ones(100),), free_energy=false)

Inference results:
-----------------------------------------
x = NormalWeightedMeanPrecision{Float64}(xi=6400.0, w=3200.01)


In [48]:
mean_var(result.posteriors[:x])

(1.999993750019531, 0.00031249902344055174)